In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../../../')
sys.path.append('../')

import model_log
from model_log import results
from model_log.util import read_yaml
ex_config = read_yaml('../experiment_config.yaml')

import numpy as np
import pandas as pd
import pickle

import ipysheet
import ipywidgets as widgets

import os
import pathlib

import matplotlib as mpl
import matplotlib.pyplot as plt

# Load experiments

In [ ]:
experiment_name = results.get_default_experiment_name();

all_configs = results.get_default_all_configs(experiment_name, model_root='../models');

experiments  = model_log.sacred_manager.match_config_filters_to_fold_ids(experiment_name, all_configs)

# Plot Metrics across all folds

In [ ]:
metrics = ['training_time', 'pred_testing_mse']
metric_headers = ['Time Taken /s', 'MSE']

In [ ]:
df, raw_df = results.get_ordered_table(
    experiment_name, metrics[:],
    group_by=['name'],
    results_by=[],
    num_folds=ex_config['NUM_FOLDS'],
    return_raw=True
)

df.columns.set_names(
    ['Metrics'],
    inplace=True
)

df = df.rename(columns={
    'training_time_score': 'Training Time',
    'pred_testing_mse_score': 'MSE'
})

df.index.set_names(
    ['Name'],
    inplace=True
)

df

# Plot results across all folds

In [ ]:
all_configs = results.get_default_all_configs(
    experiment_name, 
    model_root='../models',
    drop_fold=False
);


experiment_to_plot = results.get_names_that_match_filter(
    all_configs,
    {
        'config.name': 'template',
    },
)

experiments  = model_log.sacred_manager.match_config_filters_to_experiment_ids(
    experiment_name, 
    all_configs
)

In [ ]:
#load results dict
results_dict = {}
name = all_configs[list(all_configs.keys())[0]]['config.name']
for k, i in experiments.items():
    try:
        exp_name = '{name}_{_id}'.format(name=name, _id=i)
        print('loading: ', exp_name)
        _results = pickle.load(open('../results/{name}.pickle'.format(name=exp_name), "rb" ))
        results_dict[k] = _results
    except:
        print('Could not load: ', k)

In [ ]:
fig, axes = plt.subplots(ex_config['NUM_FOLDS'], figsize=(20, 10))

pred_data = pickle.load(open('../data/pred_data.pickle', "rb" ))

for fold in range(ex_config['NUM_FOLDS']):
    train_data = pickle.load(open('../data/train_data_{fold}.pickle'.format(fold=fold), "rb" ))
    test_data = pickle.load(open('../data/test_data_{fold}.pickle'.format(fold=fold), "rb" ))

    res = results.get_names_that_match_filter(all_configs, {
        'config.fold': fold,
        'config.name': 'template'
    })[0]
    result = results_dict[res]
    
    axes[fold].plot(pred_data['vis']['X'], result['pred']['vis']['mean'])
    axes[fold].scatter(train_data['X'], train_data['Y'])

In [ ]:
chkhjsdfsdfsd clean me